# PRÁCTICA 1

Comentarios publicados oficialmente en el moodle:
1) Para dos instancias de distinto tamaño (por ejemplo, de 100 y 200 nodos), ejecutar el algoritmo constructivo del vecino más cercano varias veces partiendo de nodos diferentes, comparando, para cada instancia, los resultados obtenidos (tiempo y resultado). 

2) Para las instancias generadas en el apartado 1, y partiendo de los mismos nodos, aplicar el algoritmo implementado comparando de nuevo tiempos y resultados.

3) Para las instancias resueltas, comparar los resultados obtenidos en 1) y 2). ¿Se puede sacar alguna conclusión sobre la eficiencia de los algoritmos comparados?

4) A cada uno de las soluciones obtenidas en 1) y 2), aplicar el algoritmo de mejora 2-opt y analizar los resultados obtenidos.

In [1]:
import pandas as pd
import numpy as np
import time

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
from pylab import rcParams

plt.style.use('classic')
rcParams['figure.figsize'] = 16, 10

# Funciones transversales

In [36]:
def define_smaple_of_datapoints(size, cmin=-100, cmax=100):
    # Semilla para la generación de números aleatorios
    np.random.seed(27)

    # size: Número total de puntos en la instancia
    size, cmin, cmax = size, cmin, cmax # la coordenada mínima y máxima, es decir, el rango de valores que pueden los costes
    data = pd.DataFrame(dict(x=[0], y=[0]))

    # Generación de datos aleatorios
    random_data = pd.DataFrame(
        (np.random.random_sample(2 * (size - 1)) * (cmax - cmin) + cmin).reshape(-1, 2),
        columns=['x', 'y']
    )

    # Concatenar los DataFrames
    data = pd.concat([data, random_data], ignore_index=True)
    return data

# Apartado 1

In [37]:
# Funciones necesarias
# Algoritmo del vecino más cercano
def nearest_neighbor(data, start_node):
    unserved = set(data.index)  # Nodos sin visitar
    current = start_node
    result_path = []

    while unserved:
        result_path.append(current)
        unserved.remove(current)
        if not unserved:
            break

        # Encontrar el nodo más cercano no visitado
        current = min(unserved, key=lambda node: np.sqrt((data.iloc[current].x - data.iloc[node].x) ** 2 +
                                                          (data.iloc[current].y - data.iloc[node].y) ** 2))

    result_path.append(start_node)  # Volver al nodo inicial
    return result_path

# Calculo de los costes de cada ruta
def estimate_cost(route, data):
    cost = 0
    for i in range(len(route) - 1):
        cost += np.sqrt((data.iloc[route[i]].x - data.iloc[route[i + 1]].x) ** 2 +
                        (data.iloc[route[i]].y - data.iloc[route[i + 1]].y) ** 2)
    return cost

# Evaluación de los costes y tiempos de ejecución
def evaluation_nearest_neighbor(data, different_nodes, list_nodes=None):
    if list_nodes is not None:
        selected_start_nodes = list_nodes
    else:
        selected_start_nodes = np.random.choice(len(data), size=different_nodes, replace=False)

    results_limited = []

    for start_node in selected_start_nodes:
        start_time = time.time()  # Iniciar temporizador
        route = nearest_neighbor(data, start_node) # Ejecución del algoritmo
        end_time = time.time()  # Finalizar temporizador
        
        cost = estimate_cost(route, data)
        exec_time = end_time - start_time  # Calcular tiempo de ejecución

        results_limited.append({
            'Start Node': start_node,
            'Total Distance': cost,
            'Execution Time (s)': exec_time,
            'Route': route
        })

    results_df_limited = pd.DataFrame(results_limited) # df con los resultados
    return results_df_limited, selected_start_nodes.tolist()


In [38]:
# Evaluación del algoritmo más cercado para 100 nodos y 200 nodos desde 10 nodos diferentes de inicio
# Configuración de los datasets:
data_100 = define_smaple_of_datapoints(size=100)
data_200 = define_smaple_of_datapoints(size=200)

# Evaluamos el algoritmo para 100 y 200 nodos
print("Algoritmo más cercano para 100 nodos y 10 diferentes puntos de inicio:")
results_df_100_apartado_1, selected_start_nodes_100 = evaluation_nearest_neighbor(data_100, 10)
display(results_df_100_apartado_1)
print("Puntos de inicio:", selected_start_nodes_100)

print("Algoritmo más cercano para 200 nodos y 10 diferentes puntos de inicio:")
results_df_200_apartado_1, selected_start_nodes_200 = evaluation_nearest_neighbor(data_200, 10)
display(results_df_200_apartado_1)
print("Puntos de inicio:", selected_start_nodes_200)

Algoritmo más cercano para 100 nodos y 10 diferentes puntos de inicio:


,Start Node,Total Distance,Execution Time (s),Route
0,61,2080.874765,0.256166,"[61, 8, 11, 77, 62, 87, 67, 10, 40, 68, 20, 22..."
1,7,2001.199302,0.245523,"[7, 2, 66, 92, 37, 25, 46, 83, 86, 70, 23, 17,..."
2,41,1974.222353,0.245057,"[41, 3, 69, 27, 96, 1, 9, 26, 16, 60, 13, 85, ..."
3,68,2151.257513,0.244840,"[68, 20, 87, 62, 77, 11, 8, 40, 10, 67, 93, 63..."
4,49,1976.148577,0.242302,"[49, 28, 52, 84, 21, 90, 57, 95, 97, 53, 6, 98..."
5,75,2056.426762,0.241423,"[75, 27, 96, 1, 9, 26, 16, 60, 13, 85, 14, 45,..."
6,45,2034.933420,0.239487,"[45, 44, 8, 11, 77, 62, 87, 67, 10, 40, 68, 20..."
7,35,2074.877216,0.240334,"[35, 59, 34, 4, 71, 12, 48, 78, 64, 29, 56, 74..."
8,79,2152.933950,0.239858,"[79, 4, 34, 59, 35, 51, 52, 28, 84, 21, 90, 57..."
9,36,2140.435714,0.240522,"[36, 32, 71, 12, 48, 78, 64, 29, 56, 74, 88, 9..."


Puntos de inicio: [61, 7, 41, 68, 49, 75, 45, 35, 79, 36]
Algoritmo más cercano para 200 nodos y 10 diferentes puntos de inicio:


,Start Node,Total Distance,Execution Time (s),Route
0,27,2841.410529,0.973463,"[27, 166, 118, 151, 167, 170, 37, 92, 149, 66,..."
1,47,2826.960888,0.974067,"[47, 173, 161, 183, 182, 42, 128, 16, 175, 26,..."
2,169,2635.404924,0.963697,"[169, 6, 98, 72, 103, 17, 23, 70, 86, 7, 2, 66..."
3,132,2534.919187,0.962259,"[132, 22, 100, 189, 108, 93, 63, 156, 10, 67, ..."
4,164,2512.472033,0.961935,"[164, 44, 180, 45, 119, 148, 73, 109, 15, 55, ..."
5,189,2578.908739,0.972637,"[189, 100, 68, 20, 130, 157, 163, 87, 62, 77, ..."
6,42,2816.721494,0.974450,"[42, 182, 183, 161, 173, 120, 47, 122, 60, 13,..."
7,25,2839.180464,0.975975,"[25, 165, 2, 7, 86, 70, 23, 17, 168, 83, 46, 1..."
8,134,2663.916807,0.972326,"[134, 5, 58, 50, 80, 33, 179, 65, 31, 138, 127..."
9,17,2826.309828,1.030086,"[17, 23, 70, 6, 169, 98, 72, 103, 168, 83, 46,..."


Puntos de inicio: [27, 47, 169, 132, 164, 189, 42, 25, 134, 17]


# Apartado 2 y 3
Variación 3 a implementar

In [39]:
# Crear una matriz de distancias para optimización
def compute_distance_matrix(data):
    n = len(data)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.sqrt((data.iloc[i].x - data.iloc[j].x) ** 2 +
                                        (data.iloc[i].y - data.iloc[j].y) ** 2)
    return dist_matrix

# Variante Inserción Más Barata Optimizada
def insertion_cheap_variant_optimized(distance_matrix, start_nodes=[0]):
    results = []
    for start_node in start_nodes:
        start_time = time.time()
        n = len(distance_matrix)
        unserved = set(range(n))
        unserved.remove(start_node)

        current = start_node
        k = min(unserved, key=lambda node: distance_matrix[current, node])
        route = [start_node, k, start_node]
        unserved.remove(k)

        while unserved:
            best_increase = float('inf')
            best_position = None
            best_node = None

            for node in unserved:
                for i in range(len(route) - 1):
                    i_node, j_node = route[i], route[i + 1]
                    increase = (distance_matrix[i_node, node] +
                                distance_matrix[node, j_node] -
                                distance_matrix[i_node, j_node])
                    if increase < best_increase:
                        best_increase = increase
                        best_position = i + 1
                        best_node = node

            route.insert(best_position, best_node)
            unserved.remove(best_node)

        end_time = time.time()
        cost = estimate_cost(route, distance_matrix)
        execution_time = end_time - start_time

        results.append({
            'Start Node': start_node,
            'Route': route,
            'Total Distance': cost,
            'Execution Time (s)': execution_time
        })

    results_df = pd.DataFrame(results)
    return results_df

# Calcular el costo de la ruta
def estimate_cost(route, distance_matrix):
    cost = sum(distance_matrix[route[i], route[i + 1]] for i in range(len(route) - 1))
    return cost


In [40]:
# Evaluación del algoritmo más cercado para 100 nodos y 200 nodos desde 10 nodos diferentes de inicio
# Configuración de los datasets:
data_100_v3 = define_smaple_of_datapoints(size=100)
data_200_v3 = define_smaple_of_datapoints(size=200)

# Matriz de distancias
distance_matrix_100 = compute_distance_matrix(data_100_v3)
distance_matrix_200 = compute_distance_matrix(data_200_v3)

# Evaluamos el algoritmo para 100 y 200 nodos
print("Algoritmo más cercano para 100 nodos y 10 puntos de inicio definidos en el apartado 1:")
start_nodes = selected_start_nodes_100
results_df_100_apartado_2 = insertion_cheap_variant_optimized(distance_matrix_100, start_nodes)
display(results_df_100_apartado_2)

print("Algoritmo más cercano para 200 nodos y 10 puntos de inicio definidos en el apartado 1:")
start_nodes = selected_start_nodes_200
results_df_200_apartado_2 = insertion_cheap_variant_optimized(distance_matrix_200, start_nodes)
display(results_df_200_apartado_2)

Algoritmo más cercano para 100 nodos y 10 puntos de inicio definidos en el apartado 1:


,Start Node,Route,Total Distance,Execution Time (s)
0,61,"[61, 40, 10, 67, 68, 93, 63, 22, 20, 87, 62, 7...",1782.355793,0.053110
1,7,"[7, 86, 23, 17, 72, 98, 97, 53, 6, 70, 66, 50,...",1859.036816,0.051801
2,41,"[41, 69, 75, 46, 25, 83, 17, 23, 72, 98, 97, 5...",1816.518869,0.050701
3,68,"[68, 67, 10, 40, 61, 54, 88, 74, 94, 12, 71, 3...",1782.355793,0.052021
4,49,"[49, 95, 57, 90, 21, 84, 52, 51, 59, 34, 4, 79...",1825.944585,0.053011
5,75,"[75, 41, 3, 69, 9, 1, 96, 27, 33, 58, 31, 19, ...",1852.988727,0.053952
6,45,"[45, 73, 15, 55, 0, 81, 38, 89, 19, 31, 65, 82...",1786.267139,0.049291
7,35,"[35, 30, 76, 32, 36, 71, 12, 48, 78, 64, 74, 9...",1834.153230,0.051334
8,79,"[79, 49, 28, 90, 95, 57, 21, 84, 52, 51, 59, 3...",1827.764607,0.051131
9,36,"[36, 71, 12, 48, 78, 64, 74, 94, 88, 99, 54, 4...",1834.153230,0.051155


Algoritmo más cercano para 200 nodos y 10 puntos de inicio definidos en el apartado 1:


,Start Node,Route,Total Distance,Execution Time (s)
0,27,"[27, 110, 69, 41, 3, 101, 144, 124, 145, 1, 13...",2589.717258,0.397950
1,47,"[47, 161, 183, 42, 182, 16, 128, 26, 184, 193,...",2606.970182,0.405190
2,169,"[169, 72, 103, 98, 6, 97, 115, 188, 53, 117, 7...",2582.359466,0.409166
3,132,"[132, 163, 157, 130, 77, 11, 190, 39, 153, 194...",2547.876434,0.409906
4,164,"[164, 153, 39, 190, 11, 77, 62, 87, 157, 163, ...",2537.995763,0.413084
5,189,"[189, 93, 63, 108, 156, 10, 67, 40, 61, 140, 1...",2538.292118,0.407802
6,42,"[42, 183, 161, 47, 173, 120, 122, 13, 60, 85, ...",2591.867481,0.406457
7,25,"[25, 199, 75, 143, 159, 46, 83, 168, 17, 23, 1...",2570.749383,0.406571
8,134,"[134, 50, 58, 80, 149, 66, 70, 117, 53, 188, 1...",2590.106729,0.406645
9,17,"[17, 168, 23, 83, 159, 46, 143, 75, 199, 25, 1...",2581.340895,0.410414


# Apartado 4

In [44]:
import numpy as np
import pandas as pd
import time

# Crear una matriz de distancias para optimización
def compute_distance_matrix(data):
    n = len(data)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.sqrt((data.iloc[i].x - data.iloc[j].x) ** 2 +
                                        (data.iloc[i].y - data.iloc[j].y) ** 2)
    return dist_matrix

# Algoritmo de mejora 2-opt
def two_opt(route, distance_matrix):
    best_route = route[:]
    best_cost = estimate_cost(best_route, distance_matrix)
    improved = True

    while improved:
        improved = False
        for i in range(1, len(route) - 2):
            for j in range(i + 1, len(route) - 1):
                new_route = best_route[:i] + best_route[i:j + 1][::-1] + best_route[j + 1:]
                new_cost = estimate_cost(new_route, distance_matrix)
                if new_cost < best_cost:
                    best_route = new_route
                    best_cost = new_cost
                    improved = True

    return best_route

# Calcular el costo de la ruta
def estimate_cost(route, distance_matrix):
    cost = sum(distance_matrix[route[i], route[i + 1]] for i in range(len(route) - 1))
    return cost

# Función para aplicar 2-opt a soluciones previas
def apply_two_opt_to_solutions(distance_matrix, solutions_df, method_name):
    results = []
    for index, row in solutions_df.iterrows():
        initial_route = row['Route']
        initial_cost = row['Total Distance']
        start_node = row['Start Node']
        
        start_time = time.time()
        improved_route = two_opt(initial_route, distance_matrix)
        end_time = time.time()

        improved_cost = estimate_cost(improved_route, distance_matrix)
        execution_time = end_time - start_time

        results.append({
            'Method': method_name,
            'Start Node': start_node,
            'Initial Route': initial_route,
            'Improved Route': improved_route,
            'Initial Cost': initial_cost,
            'Improved Cost': improved_cost,
            'Execution Time (s)': execution_time
        })

    return pd.DataFrame(results)

In [43]:
# Aplicar el 2-opt a los resultados de los apartados 1 y 2
# Calcular la matriz de distancias
data = define_smaple_of_datapoints(size=100)
distance_matrix = compute_distance_matrix(data)

# Aplicar 2-opt a las soluciones del vecino más cercano
results_2opt_apartado_1 = apply_two_opt_to_solutions(
    distance_matrix, results_df_100_apartado_1, '2-opt Vecino Más Cercano')

# Aplicar 2-opt a las soluciones de la inserción más barata
results_2opt_apartado_2 = apply_two_opt_to_solutions(
    distance_matrix, results_df_100_apartado_2, '2-opt Inserción Más Barata')

# Unir los resultados en un solo DataFrame
all_results_2opt = pd.concat([results_2opt_apartado_1, results_2opt_apartado_2], ignore_index=True)
display(all_results_2opt)

,Method,Start Node,Initial Route,Improved Route,Initial Cost,Improved Cost,Execution Time (s)
0,2-opt Vecino Más Cercano,61,"[61, 8, 11, 77, 62, 87, 67, 10, 40, 68, 20, 22...","[61, 91, 43, 56, 74, 78, 64, 29, 81, 38, 24, 1...",2080.874765,1680.659594,0.272754
1,2-opt Vecino Más Cercano,7,"[7, 2, 66, 92, 37, 25, 46, 83, 86, 70, 23, 17,...","[7, 25, 46, 83, 86, 70, 23, 17, 72, 98, 6, 53,...",2001.199302,1674.262409,0.330489
2,2-opt Vecino Más Cercano,41,"[41, 3, 69, 27, 96, 1, 9, 26, 16, 60, 13, 85, ...","[41, 69, 27, 96, 1, 9, 16, 26, 55, 15, 91, 56,...",1974.222353,1730.427202,0.329390
3,2-opt Vecino Más Cercano,68,"[68, 20, 87, 62, 77, 11, 8, 40, 10, 67, 93, 63...","[68, 10, 67, 87, 62, 77, 39, 11, 40, 61, 8, 44...",2151.257513,1691.509419,0.337007
4,2-opt Vecino Más Cercano,49,"[49, 28, 52, 84, 21, 90, 57, 95, 97, 53, 6, 98...","[49, 84, 21, 90, 57, 95, 97, 53, 6, 98, 72, 17...",1976.148577,1729.164755,0.267671
5,2-opt Vecino Más Cercano,75,"[75, 27, 96, 1, 9, 26, 16, 60, 13, 85, 14, 45,...","[75, 69, 41, 3, 47, 42, 60, 13, 85, 39, 14, 45...",2056.426762,1713.153084,0.376863
6,2-opt Vecino Más Cercano,45,"[45, 44, 8, 11, 77, 62, 87, 67, 10, 40, 68, 20...","[45, 14, 26, 16, 60, 13, 85, 47, 42, 9, 1, 96,...",2034.933420,1692.527304,0.202573
7,2-opt Vecino Más Cercano,35,"[35, 59, 34, 4, 71, 12, 48, 78, 64, 29, 56, 74...","[35, 59, 51, 52, 28, 84, 21, 90, 57, 95, 97, 5...",2074.877216,1723.656389,0.268892
8,2-opt Vecino Más Cercano,79,"[79, 4, 34, 59, 35, 51, 52, 28, 84, 21, 90, 57...","[79, 4, 34, 59, 49, 24, 18, 82, 65, 5, 58, 50,...",2152.933950,1780.568730,0.269030
9,2-opt Vecino Más Cercano,36,"[36, 32, 71, 12, 48, 78, 64, 29, 56, 74, 88, 9...","[36, 32, 76, 30, 35, 51, 52, 28, 84, 21, 90, 5...",2140.435714,1740.893631,0.268557
